In [1]:
!python -V

Python 3.10.6


In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.15.0'

In [3]:
#Invoke-WebRequest -Uri "https://www.westernpest.com/wp-content/uploads/honeybee-with-pollen.jpg" -OutFile "./honeybee-with-pollen.jpg"

'Invoke-WebRequest' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
model = keras.models.load_model('bees-wasps.h5')

In [18]:
from tensorflow.keras.preprocessing import image
# Load the image
img_path = 'honeybee-with-pollen.jpg'  # replace with the path to your image file
img = image.load_img(img_path, target_size=(150, 150))
# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # the model was trained on inputs that were normalized in the range [0, 1]

# Use the model to predict the class of the image
prediction = model.predict(img_tensor)

# Print the prediction
if prediction < 0.5:
    print("The image is predicted to be a bee.")
else:
    print("The image is predicted to be a wasp.")

1/1 [==============================] - 0s 99ms/step
The image is predicted to be a bee.


In [19]:
prediction

array([[0.45456693]], dtype=float32)

In [13]:
#Invoke-WebRequest -Uri "https://en.m.wikipedia.org/wiki/File:European_wasp_white_bg02.jpg" -OutFile "./European_wasp_white_bg02.jpg"

In [20]:
img_path = 'European_wasp_white_bg02.jpg'  # replace with the path to your image file
img = image.load_img(img_path, target_size=(150, 150))
# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # the model was trained on inputs that were normalized in the range [0, 1]

# Use the model to predict the class of the image
prediction = model.predict(img_tensor)

# Print the prediction
if prediction < 0.5:
    print("The image is predicted to be a bee.")
else:
    print("The image is predicted to be a wasp.")

1/1 [==============================] - 0s 37ms/step
The image is predicted to be a wasp.


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bee-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Ahmad\AppData\Local\Temp\tmpqm8jcobw\assets


INFO:tensorflow:Assets written to: C:\Users\Ahmad\AppData\Local\Temp\tmpqm8jcobw\assets


In [24]:
import os
os.path.getsize("bee-model.tflite") / (1024*1024)

42.787742614746094

In [3]:
import tensorflow.lite as tflite
interpreter = tflite.Interpreter(model_path='bee-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

13

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [5]:
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
img = download_image(url)
target_size = (150, 150)
prep_img = prepare_image(img, target_size)

In [7]:
type(prep_img)

PIL.Image.Image

In [8]:
pix = prep_img.load()

# Get the RGB values of the first pixel
r, g, b = pix[0, 0]

print(f"The R value of the first pixel is {r}")

The R value of the first pixel is 241


In [9]:
normalized_r = r / 255
print(normalized_r)

0.9450980392156862


In [21]:
# Load the image
img_path = 'test.jpeg'  # replace with the path to your image file
img = image.load_img(img_path, target_size=(150, 150))

# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # the model was trained on inputs that were normalized in the range [0, 1]

# Use the .h5 model to predict the class of the image
prediction_h5 = model.predict(img_tensor)

# Print the prediction
print(prediction_h5)
if prediction_h5 < 0.5:
    print("The image is predicted to be a bee.")
else:
    print("The image is predicted to be a wasp.")

# Now use the .tflite model
interpreter = tflite.Interpreter(model_path='bee-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, img_tensor)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

# Print the prediction
print(preds)
if preds < 0.5:
    print("The image is predicted to be a bee.")
else:
    print("The image is predicted to be a wasp.")


1/1 [==============================] - 0s 40ms/step
[[0.65921396]]
The image is predicted to be a wasp.
[[0.6592137]]
The image is predicted to be a wasp.


In [22]:
import requests
from PIL import Image
from io import BytesIO

# URL of the image
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

# Send a HTTP request to the URL of the image
response = requests.get(url)

# Open the URL image as a PIL image object
img = Image.open(BytesIO(response.content))

# Preprocess the image
target_size = (150, 150)
img = prepare_image(img, target_size) # Resize the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # Normalize to [0,1]

# Now use the .tflite model
interpreter = tflite.Interpreter(model_path='bee-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, img_tensor)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

# Print the prediction
print(preds)
if preds < 0.5:
    print("The image is predicted to be a bee.")
else:
    print("The image is predicted to be a wasp.")


[[0.6592137]]
The image is predicted to be a wasp.


To build the docker image for hw#09 Q6:

```bash
docker build -t ml-zoom-09 .
```

To run the docker container for hw#09 Q6:

```bash
docker run -p 9000:8080 -it ml-zoom-09
```

To test the lambda function in the docker container locally:

```bash
$ curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{"url":"https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"}'
```

Output:

```json
{"Bee": 0.5546649396419525, "Wasp": 0.4453350603580475}
```